In [ ]:
!pip install torch==1.12.0 torchvision --extra-index-url https://download.pytorch.org/whl/cu113
# Install MMCV
!pip install openmim
!mim install mmcv-full==1.6.0
!git clone https://github.com/open-mmlab/mmsegmentation.git

%cd mmsegmentation
!pip install -e .

import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
import mmcv
import os.path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |██████████████▌                 | 834.1 MB 1.3 MB/s eta 0:13:16tcmalloc: large alloc 1147494400 bytes == 0x3a784000 @  0x7f5b1c3dd615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |██████████████████▍             | 1055.7 MB 1.3 MB/s eta 0:10:25tcmalloc: large alloc 1434370048 bytes == 0x7edda000 @  0x7f5b1c3dd615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████████████▎        | 

In [ ]:
!pip install spams
!pip install staintools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 24.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for spams: filename=spams-2.6.5.4-cp37-cp37m-linux_x86_64.whl size=3236588 sha256=744f38ddc0d465d06f01ee2c4eba1b082bdadf75cbcf2235878966f561a597c6
  Stored in directory: /root/.cache/pip/wheels/5e/92/17/e718e0e26bfcdd7e81afcf874ac2bc599dd7dd7f3ad78f9d76
Successfully built spams


In [ ]:
%cd /content/mmsegmentation/mmseg/datasets/pipelines

/content/mmsegmentation/mmseg/datasets/pipelines


In [ ]:
%%bash

cat <<EOT >> ./pixelNthick.py
from ..builder import PIPELINES

@PIPELINES.register_module()
class MyTransform:

    def __call__(self, image, domain_pixel_size=0.4, target_pixel_size=0.4, domain_tissue_thickness=4, target_tissue_thickness=4, alpha=0.15):  
      # Augment tissue thickness
      tissue_thickness_scale_factor = target_tissue_thickness - domain_tissue_thickness
      image_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(np.float32)
      image_hsv[:, :, 1] *= (1 + (alpha * tissue_thickness_scale_factor))
      image_hsv[:, :, 2] *= (1 - (alpha * tissue_thickness_scale_factor))
      image_hsv = image_hsv.astype(np.uint8)
      image_scaled = cv2.cvtColor(image_hsv, cv2.COLOR_HSV2RGB)
      
      # Standardize luminosity
      image_scaled = staintools.LuminosityStandardizer.standardize(image_scaled)

      # Augment pixel size
      pixel_size_scale_factor = domain_pixel_size / target_pixel_size
      image_resized = cv2.resize(
          image_scaled,
          dsize=None,
          fx=pixel_size_scale_factor,
          fy=pixel_size_scale_factor,
          interpolation=cv2.INTER_CUBIC
      )
      image_resized = cv2.resize(
          image_resized,
          dsize=(
              image.shape[1],
              image.shape[0]
          ),
          interpolation=cv2.INTER_CUBIC
      )
      
      # Standardize luminosity
      image = staintools.LuminosityStandardizer.standardize(image)
      image_augmented = staintools.LuminosityStandardizer.standardize(image_resized)
      
      return (np.array(image_augmented).astype(np.float32))/255

EOT

In [ ]:
from glob import glob
import numpy as np
import cv2
import os
from sklearn.model_selection import StratifiedKFold

Fold = 10
all_mask_files = glob("/content/drive/MyDrive/hubmap-organ-segmentation/masks/*")
masks = []
num_mask = np.zeros((6,Fold))

!mkdir splits

split = list(StratifiedKFold(n_splits=Fold, shuffle=True, random_state=2022).split(all_mask_files, [0]*len(all_mask_files)))
for fold, (train_idx, valid_idx) in enumerate(split):
    with open(f"/content/drive/MyDrive/hubmap-organ-segmentation/splits/fold_{fold}.txt", "w") as f:
        for idx in train_idx:
            f.write(os.path.basename(all_mask_files[idx])[:-4] + "\n")
    with open(f"/content/drive/MyDrive/hubmap-organ-segmentation/splits/valid_{fold}.txt", "w") as f:
        for idx in valid_idx:
            f.write(os.path.basename(all_mask_files[idx])[:-4] + "\n")

In [ ]:
%cd 

In [ ]:
%%bash

cat <<EOT >> ./config2.py

_base_ = ['/content/mmsegmentation/configs/segformer/segformer_mit-b0_512x512_160k_ade20k.py']

# dataset settings
dataset_type = 'CustomDataset'
data_root = '/content/drive/MyDrive/hubmap-organ-segmentation/'
classes = ['background', 'kidney', 'prostate', 'largeintestine', 'spleen', 'lung']
palette = [[0,0,0], [255,0,0], [0,255,0], [0,0,255], [255,255,0], [255,0,255]]

# img_norm_cfg = dict(mean=[196.869, 190.186, 194.802], std=[63.010, 66.765, 65.745], to_rgb=True)
img_norm_cfg = dict(mean=[0.77907741, 0.75481993, 0.77184585], std=[0.25398408, 0.26743021, 0.26288133])
size = 256

train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(size, size), keep_ratio=True),
    dict(type='MyTransform'),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size=(size, size), pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(size, size),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size=(size, size), pad_val=0, seg_pad_val=255),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=8,
    workers_per_gpu=4,
    train=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='train',
        ann_dir='masks',
        img_suffix=".png",
        seg_map_suffix='.png',
        split="splits/fold_0.txt",
        classes=classes,
        palette=palette,
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='train',
        ann_dir='masks',
        img_suffix=".png",
        seg_map_suffix='.png',
        split="splits/valid_0.txt",
        classes=classes,
        palette=palette,
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        data_root=data_root,
        test_mode=True,
        img_dir='train',
        ann_dir='masks',
        img_suffix=".png",
        seg_map_suffix='.png',
        classes=classes,
        palette=palette,
        pipeline=test_pipeline))

checkpoint = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segformer/mit_b5_20220624-658746d9.pth'  # noqa

# model settings
model = dict(
    pretrained=checkpoint,
    backbone=dict(
        embed_dims=64, num_heads=[1, 2, 5, 8], num_layers=[3, 6, 40, 3]),
    decode_head=dict(in_channels=[64, 128, 320, 512]))

EOT

In [ ]:
!python ./tools/train.py ./config2.py

2022-08-29 01:49:42,756 - mmseg - INFO - Multi-processing start method is `None`
2022-08-29 01:49:42,756 - mmseg - INFO - OpenCV num_threads is `2
2022-08-29 01:49:42,793 - mmseg - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.12.0+cu113
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC arc

In [ ]:
%cd /content/mmsegmentation

/content/mmsegmentation


In [ ]:
import numpy as np
a = np.zeros(10)
type(a)

numpy.ndarray